In [1]:
import pandas as pd
import sys
import os
sys.path.insert(0, '../../../')

from notebooks.utils import _ALEXA_DATA_PATH, load_node_features, load_level_data, create_overlap_nodes, export_model_as_feature
from train import run_experiment

2022-01-12 01:46:51.500008: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load audience overlap edges for level 3

In [2]:
level = 3
audience_overlap_sites = load_level_data(os.path.join(_ALEXA_DATA_PATH, 'corpus_2020_audience_overlap_sites_scrapping_result.json'), level=level)
audience_overlap_sites_NODES = create_overlap_nodes(audience_overlap_sites)

print(audience_overlap_sites_NODES[:5])

01-12 01:46:54 notebooks.utils INFO     Loaded 26573 nodes with records level <= 3 and child size:128548


[('crooked.com', 'votesaveamerica.com'), ('crooked.com', 'art19.com'), ('crooked.com', 'promocodeportal.com'), ('crooked.com', 'mediamatters.org'), ('crooked.com', 'actblue.com')]


In [3]:
edge_df = pd.DataFrame(audience_overlap_sites_NODES, columns=['source', 'target'])

edge_df.head()

,source,target
0,crooked.com,votesaveamerica.com
1,crooked.com,art19.com
2,crooked.com,promocodeportal.com
3,crooked.com,mediamatters.org
4,crooked.com,actblue.com


### Find all unique nodes in edges

In [4]:
nodes_in_edges = list(set(edge_df.source.unique().tolist() + edge_df.target.unique().tolist()))
print('Number of unique nodes in edges:', len(nodes_in_edges), 'Sample:', nodes_in_edges[:5])

Number of unique nodes in edges: 67351 Sample: ['rflora.com', 'itc.gov.hk', 'dometours.com', 'understandingislam.today', 'tepsa.eu']


### 1. Load all node features

In [5]:
node_features_df = load_node_features()
node_features_df = node_features_df.set_index('site')
node_features_df.head()

,alexa_rank,daily_pageviews_per_visitor,daily_time_on_site,total_sites_linking_in,bounce_rate
site,,,,,
whistleblowersandrelators.com,NaN,NaN,NaN,NaN,NaN
geokov.com,2238341.0,1.0,NaN,60.0,0.900
trainingandfacilitation.ca,NaN,NaN,NaN,NaN,NaN
plumsolutions.com.au,1023533.0,1.0,138.0,60.0,0.813
dbdailyupdate.com,145283.0,1.7,179.0,64.0,0.756


# Subset node_features

In [6]:
node_features_df = node_features_df.loc[nodes_in_edges]

In [7]:
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67351 entries, rflora.com to studenti.it
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   40736 non-null  float64
 1   daily_pageviews_per_visitor  40746 non-null  float64
 2   daily_time_on_site           24566 non-null  float64
 3   total_sites_linking_in       63854 non-null  float64
 4   bounce_rate                  21295 non-null  float64
dtypes: float64(5)
memory usage: 3.1+ MB


### 2. Fill all missing alexa_rank and total_sites_linking_in with 0 

In [8]:
node_features_df.alexa_rank = node_features_df.alexa_rank.fillna(0)
node_features_df.total_sites_linking_in = node_features_df.total_sites_linking_in.fillna(0)
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67351 entries, rflora.com to studenti.it
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   alexa_rank                   67351 non-null  float64
 1   daily_pageviews_per_visitor  40746 non-null  float64
 2   daily_time_on_site           24566 non-null  float64
 3   total_sites_linking_in       67351 non-null  float64
 4   bounce_rate                  21295 non-null  float64
dtypes: float64(5)
memory usage: 3.1+ MB


### 3. Normalizing features

In [9]:
import math

node_features_df['normalized_alexa_rank'] = node_features_df['alexa_rank'].apply(lambda x: 1/x if x else 0)
node_features_df['normalized_total_sites_linked_in'] = node_features_df['total_sites_linking_in'].apply(lambda x: math.log2(x) if x else 0)

# Create Graph

In [10]:
import stellargraph as sg

G = sg.StellarGraph(nodes=node_features_df.loc[nodes_in_edges, ['normalized_alexa_rank', 'normalized_total_sites_linked_in']], edges=edge_df)

print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 67351, Edges: 128582

 Node types:
  default: [67351]
    Features: float32 vector, length 2
    Edge types: default-default->default

 Edge types:
    default-default->default: [128582]
        Weights: all 1 (default)
        Features: none


# Unsupervised Attrib2Vec

In [11]:
from stellargraph.mapper import Attri2VecLinkGenerator, Attri2VecNodeGenerator
from stellargraph.layer import Attri2Vec, link_classification
from stellargraph.data import UnsupervisedSampler

from tensorflow import keras

# 1. Specify the other optional parameter values: root nodes, the number of walks to take per node, the length of each walk, and random seed.

nodes = list(G.nodes())
number_of_walks = 1
length = 5

# 2. Create the UnsupervisedSampler instance with the relevant parameters passed to it.
unsupervised_samples = UnsupervisedSampler(G, nodes=nodes, length=length, number_of_walks=number_of_walks)

# 3. Create a node pair generator:
batch_size = 50
epochs = 4
num_samples = [10, 5]

generator = Attri2VecLinkGenerator(G, batch_size)
train_gen = generator.flow(unsupervised_samples)

layer_sizes = [128]
attri2vec = Attri2Vec(layer_sizes=layer_sizes, generator=generator, bias=False, normalize=None)

# Build the model and expose input and output sockets of attri2vec, for node pair inputs:
x_inp, x_out = attri2vec.in_out_tensors()

prediction = link_classification(output_dim=1, output_act="sigmoid", edge_embedding_method="ip")(x_out)

model = keras.Model(inputs=x_inp, outputs=prediction)

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [13]:
history = model.fit(train_gen, epochs=epochs, verbose=2, use_multiprocessing=False, workers=1, shuffle=True)

Epoch 1/4
10777/10777 - 691s - loss: 0.7204 - binary_accuracy: 0.5787 - 691s/epoch - 64ms/step
Epoch 2/4
10777/10777 - 678s - loss: 0.7541 - binary_accuracy: 0.5690 - 678s/epoch - 63ms/step
Epoch 3/4
10777/10777 - 650s - loss: 0.7548 - binary_accuracy: 0.5703 - 650s/epoch - 60ms/step
Epoch 4/4
10777/10777 - 747s - loss: 0.7555 - binary_accuracy: 0.5736 - 747s/epoch - 69ms/step


In [14]:
x_inp_src = x_inp[0]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

node_gen = Attri2VecNodeGenerator(G, batch_size).flow(node_features_df.index.tolist())
node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=1)

embeddings_wv = dict(zip(node_features_df.index.tolist(), node_embeddings.tolist()))

print('Sample:', embeddings_wv['crooked.com'][:10], len(embeddings_wv['crooked.com']))

1348/1348 [==============================] - 1s 762us/step
Sample: [4.549659069861959e-16, 2.0329970415633537e-22, 1.4219621391475812e-07, 1.194521956904282e-07, 3.5874450077244546e-09, 1.7870845428480386e-11, 2.1038040537596796e-16, 1.1590037263253805e-10, 3.8452368459479525e-13, 8.100271321753727e-12] 128


# Export embeddings as feature

In [15]:
export_model_as_feature(embeddings_wv, f'attrib2vec_audience_overlap_level_{level}_epochs_{epochs}')

'/Users/panayot/Documents/News-Media-Peers/data/acl2020/features/attrib2vec_audience_overlap_level_3_epochs_4.json'

In [16]:
run_experiment(features=f'attrib2vec_audience_overlap_level_{level}_epochs_{epochs}')

+------+---------------------+---------------+--------------------+----------------------------------------------+
| task | classification_mode | type_training | normalize_features |                   features                   |
+------+---------------------+---------------+--------------------+----------------------------------------------+
| fact |  single classifier  |    combine    |        True        | attrib2vec_audience_overlap_level_3_epochs_4 |
+------+---------------------+---------------+--------------------+----------------------------------------------+


01-12 09:50:09 train        INFO     Start training...
01-12 09:50:09 train        INFO     Fold: 0
01-12 09:50:14 train        INFO     Fold: 1
01-12 09:50:17 train        INFO     Fold: 2
01-12 09:50:19 train        INFO     Fold: 3
01-12 09:50:22 train        INFO     Fold: 4


+------+---------------------+---------------+--------------------+----------------------------------------------+--------------------+-------------------+-------------------+--------------------+
| task | classification_mode | type_training | normalize_features |                   features                   |      Macro-F1      |      Accuracy     |  Flip error-rate  |        MAE         |
+------+---------------------+---------------+--------------------+----------------------------------------------+--------------------+-------------------+-------------------+--------------------+
| fact |  single classifier  |    combine    |        True        | attrib2vec_audience_overlap_level_3_epochs_4 | 34.549749416566925 | 54.59837019790454 | 14.78463329452852 | 0.6018626309662398 |
+------+---------------------+---------------+--------------------+----------------------------------------------+--------------------+-------------------+-------------------+--------------------+
